In [1]:
from __future__ import division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyomo.environ as pyo, numpy as np, pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory


geradores = pd.read_excel('input.xlsx', sheet_name='geradores')
demanda   = pd.read_excel('input.xlsx', sheet_name='demanda')


In [2]:
demanda

,t,dem
0,0,500
1,1,355
2,2,480
3,3,650
4,4,758
5,5,791
6,6,885


In [3]:
# Entrada de dados:
#
# Geradores
a = geradores.A.values
b = geradores.B.values
c = geradores.C.values
cos = geradores.Custo.values
pmin = geradores.Pmin.values
pmax = geradores.Pmax.values
ur = geradores.UR.values
dr = geradores.DR.values
#cos = geradores.Custo.to_list()
#pmin = geradores.Pmin.to_list()
#pmax = geradores.Pmax.to_list()
#ur = geradores.UR.to_list()
#dr = geradores.DR.to_list()

# Demanda
dem = demanda.dem.values
t = demanda.t.to_list()

Ng = len(geradores)
Nd = len(dem)
Nd

7

In [4]:
model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), range(Nd), bounds=(0,None))
model.Pd = pyo.Var(range(Nd))
Pg = model.Pg
Pd = model.Pd

In [5]:
#objetivo
model.obj = pyo.Objective(expr= sum([a[g]*cos[g] 
                                    + b[g]*cos[g]*Pg[g,t] 
                                    + c[g]*cos[g]*Pg[g,t]*Pg[g,t] 
                                    for g in range(Ng) 
                                    for t in range(Nd)]),
                                     sense=minimize)

In [6]:
# balanco:
model.balanco = ConstraintList()
#sum_Pg = 0
for t in range(Nd):
    sum_Pg = sum(Pg[g,t] for g in range(Ng))
    model.balanco.add(expr= sum_Pg == Pd[t])

In [7]:
"""
# limites geração:
model.limger = pyo.ConstraintList()
for g in range(Ng):
    for t in range(Nd):
        model.limger.add(inequality(pmin[g], Pg[g,t], pmax[g]))
"""

In [8]:
opt = SolverFactory('couenne', executable='C:\\couenne\\bin\\couenne.exe')
opt.solve(model)

model.pprint()

6 Set Declarations
    Pd_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {0, 1, 2, 3, 4, 5, 6}
    Pg_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                : Size : Members
        None :     2 : Pg_index_0*Pg_index_1 :   21 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6)}
    Pg_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    Pg_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {0, 1, 2, 3, 4, 5, 6}
    balanco_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    limger_index :

In [11]:
for g in range(Ng):
    for t in range(Nd):
        print(f"Pg[{g},{t}] = {pyo.value(Pg[g,t])}")


Pg[0,0] = 150.0
Pg[0,1] = 150.0
Pg[0,2] = 150.0
Pg[0,3] = 150.0
Pg[0,4] = 150.0
Pg[0,5] = 150.0
Pg[0,6] = 150.0
Pg[1,0] = 100.0
Pg[1,1] = 100.0
Pg[1,2] = 100.0
Pg[1,3] = 100.0
Pg[1,4] = 100.0
Pg[1,5] = 100.0
Pg[1,6] = 100.0
Pg[2,0] = 50.0
Pg[2,1] = 50.0
Pg[2,2] = 50.0
Pg[2,3] = 50.0
Pg[2,4] = 50.0
Pg[2,5] = 50.0
Pg[2,6] = 50.0
